# Using Python on WRDS Platform

## Install WRDS

In [1]:
#%pip install -wrds
import wrds

## Setup WRDS connection

Now that we've installed the `wrds` module, the next step is to set up a `pgpass` file on your workstation. The `pgpass` file includes your WRDS username and password so that you do not need to enter them each time you wish to connect to WRDS within Python. With the `wrds` module, creating this file is easy!

First, start python on your workstation. Then following these steps (`wrds_username` is your own WRDS username, the same as your login to the WRDS website) and you will be prompted for your WRDS username and password on your first login:

In [2]:
db = wrds.Connection(wrds_username='valexeev')
db.create_pgpass_file()

Enter your WRDS username [valexeev]: valexeev
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
Loading library list...
Done


You should be able to connect from then on without needing to do so. Test this by disconnecting and reconnecting, using the following:

In [3]:
db.close()
db = wrds.Connection(wrds_username='valexeev')

Loading library list...
Done


### Managing your Connections

WRDS users are permitted up to 5 simultaneous connections to our Postgres data backend. For Python users, that means you may use `wrds.Connection()` up to five times before being denied additional connections. The best way to manage this, is to properly close out your connection to WRDS once you are done with it. With the Python wrds module, you simply use the `close()` method like so:

In [4]:
db.close() # close the connection

# Applications

Load required libraries and establish the connection:

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats

import wrds
db = wrds.Connection(wrds_username='valexeev')

## Fama-French factors

In [3]:
ff = db.get_table(library='ff', table='factors_monthly')
ff = ff[['date','smb','hml']]
ff['date']=ff['date']+MonthEnd(0) # convert the first date of a month to the end date of a month, e.g., 01 Jan 2023 becomes 31 Jan 2023
ff

,date,smb,hml
0,1926-07-31,-0.0256,-0.0243
1,1926-08-31,-0.0117,0.0382
2,1926-09-30,-0.0140,0.0013
3,1926-10-31,-0.0009,0.0070
4,1926-11-30,-0.0010,-0.0051
...,...,...,...
1154,2022-09-30,-0.0079,0.0006
1155,2022-10-31,0.0009,0.0805
1156,2022-11-30,-0.0340,0.0138
1157,2022-12-31,-0.0068,0.0132


# Limiting the Number of Records Returned

When working with large data sources, it is important to begin your research with small subsets of the data you eventually want to query. Limiting the number of returned records (also called observations) is essential while developing your code, as queries that involve large date ranges or query a large number of variables (column fields) could take a long time and generate large output files.

Generally, until you are sure that you're getting exactly the data you're looking for, you should limit the number of observations returned to a sensible maximum such as 10 or 100. Remember, much of the data provided at WRDS is huge! It is highly recommended to develop your code using such a limit, then simply remove that limit when you are ready to run your final code.

**IMPORTANT:** This is especially important if you are running Python locally from your computer, as the returned query output data is downloaded from WRDS to your computer. Even if you have a fast computer, a slow or intermittent Internet connection could cripple your research if you don't perform your queries carefully.

In [4]:
ff = db.get_table(library='ff', table='factors_monthly', obs=100)
ff

,date,mktrf,smb,hml,rf,year,month,umd,dateff
0,1926-07-01,0.0296,-0.0256,-0.0243,0.0022,1926.0,7.0,NaN,1926-07-31
1,1926-08-01,0.0264,-0.0117,0.0382,0.0025,1926.0,8.0,NaN,1926-08-31
2,1926-09-01,0.0036,-0.0140,0.0013,0.0023,1926.0,9.0,NaN,1926-09-30
3,1926-10-01,-0.0324,-0.0009,0.0070,0.0032,1926.0,10.0,NaN,1926-10-30
4,1926-11-01,0.0253,-0.0010,-0.0051,0.0031,1926.0,11.0,NaN,1926-11-30
...,...,...,...,...,...,...,...,...,...
95,1934-06-01,0.0264,-0.0223,-0.0296,0.0001,1934.0,6.0,-0.0096,1934-06-30
96,1934-07-01,-0.1096,-0.0695,-0.1070,0.0001,1934.0,7.0,0.0871,1934-07-31
97,1934-08-01,0.0558,0.0537,0.0009,0.0001,1934.0,8.0,0.0248,1934-08-31
98,1934-09-01,-0.0023,-0.0152,-0.0120,0.0001,1934.0,9.0,0.0006,1934-09-29


# Methods in the wrds Module

The `wrds` module supports several different methods of accessing WRDS data.

- `db.close()`
- `db.connection()`
- `db.describe_table()`
- `db.get_table()`
- `db.list_tables()`
- `db.raw_sql()`
- `db.get_row_count()`
- `db.list_libraries()`

For an online description of each, use the `help()` function as follows:

`help(db.get_table)`

Data at WRDS is organized in a hierarchical manner by vendor (e.g., **crsp** for CRSP data or **ff** for Fama-French data), referred to at the top-level as *libraries*. Each library contains a number of component *tables* or datasets (e.g., *factors_daily* or *factors_monthly*) which contain the actual data in tabular format, with column headers as variables (such as *data*, *mktrf*, *smb*, etc).

You can analyze the structure of the data through its metadata using the `wrds` module, as outlined in the following steps:

1. List all available libraries at WRDS using `list_libraries()`
2. Select a library to work with, and list all available datasets within that library using `list_tables()`
3. Select a dataset, and list all available variables (column headers) within that dataset using `describe_table()`

**NOTE:** When referencing library and dataset names, you must use **all lowercase**.

Alternatively, a comprehensive list of all WRDS libraries is available at the [Dataset List](https://wrds-www.wharton.upenn.edu/pages/get-data/). This resource provides a listing of each library, their component datasets and variables, as well as a tabular database preview feature, and is helpful in establishing the structure of the data you're looking for in an easy manner from a Web browser.

In [10]:
db.list_libraries()

['aha_sample',
 'ahasamp',
 'audit',
 'audit_audit_comp',
 'audit_common',
 'auditsmp',
 'auditsmp_all',
 'block',
 'block_all',
 'boardex',
 'boardex_na',
 'boardex_trial',
 'boardsmp',
 'calcbench_income_tax',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'ciqsamp',
 'ciqsamp_common',
 'ciqsamp_transcripts',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_filings',
 'comp_na_annual_all',
 'comp_na_daily_all',
 'comp_na_monthly_all',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'comp_snapshot',
 'compa',
 'compb',
 'compm',
 'compsamp_snapshot',
 'compseg',
 'compsnap',
 'contrib',
 'contrib_ceo_turnover',
 'contrib_char_returns',
 'contrib_general',
 'contrib_intangible_value',
 'contrib_kpss',
 'crsp',
 'crsp_a_ccm',
 'crsp_a_stock',
 'crsp_q_mutualfunds',
 'djones',
 'djones_all',
 'dmef',
 'dmef_all',
 'doe',
 'doe_all',
 'etfg',
 'etfg_analytics',
 'etfg_constituents',
 'etfg_fund_flow',
 'etfg_industry',
 'etfg_samp',
 'etfgsam

In [6]:
db.list_tables(library='ff')

['factors_china',
 'factors_daily',
 'factors_monthly',
 'fivefactors_daily',
 'fivefactors_monthly',
 'industry12',
 'industry48',
 'liq_ps',
 'liq_sadka',
 'portfolios',
 'portfolios25',
 'portfolios_d']

In [7]:
db.describe_table(library='ff', table='factors_daily')

Approximately 25419 rows in ff.factors_daily.


,name,nullable,type,comment
0,date,True,DATE,None
1,mktrf,True,DOUBLE_PRECISION,None
2,smb,True,DOUBLE_PRECISION,None
3,hml,True,DOUBLE_PRECISION,None
4,rf,True,DOUBLE_PRECISION,None
5,umd,True,DOUBLE_PRECISION,None


In [8]:
stocknames = db.get_table(library='crsp', table='stocknames', obs=100)
stocknames.head()

,permno,namedt,nameenddt,shrcd,exchcd,siccd,ncusip,ticker,comnam,shrcls,permco,hexcd,cusip,st_date,end_date,namedum
0,10000.0,1986-01-07,1987-06-11,10.0,3.0,3990.0,68391610,OMFGA,OPTIMUM MANUFACTURING INC,A,7952.0,3.0,68391610,1986-01-07,1987-06-11,2.0
1,10001.0,1986-01-09,1993-11-21,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,None,7953.0,2.0,36720410,1986-01-09,2017-08-03,2.0
2,10001.0,1993-11-22,2008-02-04,11.0,3.0,4920.0,29274A10,EWST,ENERGY WEST INC,None,7953.0,2.0,36720410,1986-01-09,2017-08-03,2.0
3,10001.0,2008-02-05,2009-08-03,11.0,3.0,4920.0,29274A20,EWST,ENERGY WEST INC,None,7953.0,2.0,36720410,1986-01-09,2017-08-03,2.0
4,10001.0,2009-08-04,2009-12-17,11.0,3.0,4920.0,29269V10,EGAS,ENERGY INC,None,7953.0,2.0,36720410,1986-01-09,2017-08-03,2.0


In [9]:
len(stocknames)

100

## Joining Data from Separate Datasets

Data from separate datasets can be joined and analyzed together. The following example will join the Compustat Fundamentals data set (**comp.funda**) with Compustat's pricing dataset (**comp.secm**), and then query for total assets and liabilities mixed with monthly close price and shares outstanding.

In [11]:
db.list_tables(library='comp')

['aco_amda',
 'aco_imda',
 'aco_indfnta',
 'aco_indfntq',
 'aco_indfntytd',
 'aco_indsta',
 'aco_indstq',
 'aco_indstytd',
 'aco_notesa',
 'aco_notesq',
 'aco_notessa',
 'aco_notesytd',
 'aco_pnfnda',
 'aco_pnfndq',
 'aco_pnfndytd',
 'aco_pnfnta',
 'aco_pnfntq',
 'aco_pnfntytd',
 'aco_transa',
 'aco_transq',
 'aco_transsa',
 'aco_transytd',
 'adsprate',
 'asec_amda',
 'asec_imda',
 'asec_notesa',
 'asec_notesq',
 'asec_transa',
 'asec_transq',
 'bank_aacctchg',
 'bank_adesind',
 'bank_afnd1',
 'bank_afnd2',
 'bank_afnddc1',
 'bank_afnddc2',
 'bank_afntind',
 'bank_funda',
 'bank_funda_fncd',
 'bank_fundq',
 'bank_fundq_fncd',
 'bank_iacctchg',
 'bank_idesind',
 'bank_ifndq',
 'bank_ifndytd',
 'bank_ifntq',
 'bank_ifntytd',
 'bank_names',
 'bank_namesq',
 'chars',
 'co_aacctchg',
 'co_aaudit',
 'co_adesind',
 'co_adjfact',
 'co_afnd1',
 'co_afnd2',
 'co_afnddc1',
 'co_afnddc2',
 'co_afntind1',
 'co_afntind2',
 'co_ainvval',
 'co_amkt',
 'co_busdescl',
 'co_cotype',
 'co_filedate',
 'co_

In [12]:
db.describe_table(library='comp', table='funda')

Approximately 869854 rows in comp.funda.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),None
1,datadate,True,DATE,None
2,fyear,True,DOUBLE_PRECISION,None
3,indfmt,True,VARCHAR(12),None
4,consol,True,VARCHAR(2),None
...,...,...,...,...
943,au,True,VARCHAR(8),None
944,auop,True,VARCHAR(8),None
945,auopic,True,VARCHAR(1),None
946,ceoso,True,VARCHAR(1),None


In [13]:
db.describe_table(library='comp', table='secm')

Approximately 7686282 rows in comp.secm.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),None
1,iid,True,VARCHAR(3),None
2,datadate,True,DATE,None
3,tic,True,VARCHAR(8),None
4,cusip,True,VARCHAR(9),None
5,conm,True,VARCHAR(70),None
6,ajexm,True,DOUBLE_PRECISION,None
7,ajpm,True,DOUBLE_PRECISION,None
8,isalrt,True,VARCHAR(8),None
9,primiss,True,VARCHAR(1),None


### Query and join two Compustat datasets:

The code joins both datasets using a common `gvkey` identifier and date, querying IBM with a frequency of one year, resulting in 61 observations (as of 2021). Running joined queries between large datasets can require large amounts of memory and execution time. It is recommended you limit the scope of your queries to reasonable sizes when performing joins.

NOTE: For this example, you would need an active subscription to both datasets.

In [14]:
db.raw_sql("""
    SELECT a.gvkey, a.datadate, a.tic, a.conm, a.at, a.lt, b.prccm, b.cshoq
    FROM comp.funda a
    JOIN comp.secm b ON a.gvkey = b.gvkey AND a.datadate = b.datadate
    WHERE a.tic = 'IBM' AND a.datafmt = 'STD' AND a.consol = 'C' AND a.indfmt = 'INDL'
""")

,gvkey,datadate,tic,conm,at,lt,prccm,cshoq
0,006066,1962-12-31,IBM,INTL BUSINESS MACHINES CORP,2112.301,731.700,389.999567,NaN
1,006066,1963-12-31,IBM,INTL BUSINESS MACHINES CORP,2373.857,782.119,506.999353,NaN
2,006066,1964-12-31,IBM,INTL BUSINESS MACHINES CORP,3309.152,1055.072,409.499496,NaN
3,006066,1965-12-31,IBM,INTL BUSINESS MACHINES CORP,3744.917,1166.771,498.999146,NaN
4,006066,1966-12-31,IBM,INTL BUSINESS MACHINES CORP,4660.777,1338.149,371.499662,NaN
...,...,...,...,...,...,...,...,...
56,006066,2017-12-31,IBM,INTL BUSINESS MACHINES CORP,125356.000,107631.000,153.420000,922.179
57,006066,2018-12-31,IBM,INTL BUSINESS MACHINES CORP,123382.000,106453.000,113.670000,892.479
58,006066,2019-12-31,IBM,INTL BUSINESS MACHINES CORP,152186.000,131201.000,134.040000,887.110
59,006066,2020-12-31,IBM,INTL BUSINESS MACHINES CORP,155971.000,135244.000,125.880000,892.653


## Historical S&P 500 Index Constituents

UTS does not have subscription to this.

In [17]:
db.describe_table(library='crsp', table='msp500list')

Approximately 2029 rows in crsp.msp500list.


,name,nullable,type,comment
0,permno,True,DOUBLE_PRECISION,CRSP Permanent Number
1,start,True,DATE,Date when the stock included in S&P500 Index
2,ending,True,DATE,Date when the stock excluded from S&P500 Index


In [20]:
stocknames = db.get_table(library='crsp', table='msp500list', obs=100)

ProgrammingError: (psycopg2.errors.InsufficientPrivilege) permission denied for schema crsp_a_indexes

[SQL: SELECT * FROM crsp.msp500list  LIMIT 100 OFFSET 0;]
(Background on this error at: https://sqlalche.me/e/14/f405)

https://wrds-www.wharton.upenn.edu/pages/wrds-research/applications/python-replications/fama-french-factors-python/